In [1]:
import numpy as np
from docplex.mp.model import Model
import copy
import random

eps = 1e-5

In [2]:
class Graph(object):
    """
    class of graph
    """
    
    def __init__(self):
        """
        V- множество вершин
        D - множество вершин, с возможностью их сортировки по степени
        colors - нужен только для покраски по вершинам
        """
        self.V = {}
        self.D = [] 
        self._D_sorted = False
        self.used_colors = set()
        self.coloring = dict()

    def add_edge(self,v1,v2):
        """
        add pair of vertexes to V
        add each other to neigbours
        """
        self._add_vertex_to_neighbours(v1,v2)
        self._add_vertex_to_neighbours(v2,v1)
        
        return
    
    def add_vertex(self,v1):
        """
        add one vertex to V
        """
        
        if v1 in self.V: # если вершина есть в множестве, то все ок. Ее не надо добавлять
                         # МБ надо обновить ее степень?
#             v1.update_degree = len(self.neighbours(v1))
#             self.V[v1]['degree'] = self.degree(v1)
#             D[self.V[v1]['D_index'] ] = self.V[v1]['degree']
            print('vertex already in graph.Vertexes')
            
            return
        else:
            self.V[v1] = {}
            self.V[v1]['name'] = v1
            self.V[v1]['neighbours'] = set()
            self.V[v1]['degree'] = 0
            self.D.append(self.V[v1])
            self._D_sorted = False
            
            return
    
            
    def _add_vertex_to_neighbours(self,v1, v2):
        """
        add vertex v2 to neigbours of vertex v1
        """
        if v1 not in self.V:
            self.add_vertex(v1)
        if v2 in self.V[v1]['neighbours']:
            print('vertex already in neighbours')
            return
        self.V[v1]['neighbours'].add(v2)
        self.V[v1]['degree'] += 1
        self._D_sorted = False

        return
    
    def neighbours(self,v1):
        """
        print all neigbour names of chosed vertex
        """
        if v1 in self.V:
            return self.V[v1]['neighbours']
        else:
            print("no such vertex in graph")
            return None 
        
    def is_neighbour(self,vertex_name,neighbour_name):
        """
        vetrex - vertex name
        neighbout - neighbour name
        True if neighbour
        False if not neighbour
        """
        if neighbour_name in self.V[vertex_name]['neighbours']:
            return True
        return False
    
    
    def sort_vertex_by_degree(self):
        """
        sort vertexes: big degree is first
        """
        # добавить маркер сортер для массива Д для того, чтобы не сортировать сортированный Д
        if self._D_sorted :
            return
        self.D.sort(key= lambda input: input['degree'])
        self._D_sorted = True
        return

        
    
    def degree(self,v1):
        """
        return degree of chosen vertex
        """
        if v1 in self.V:
            return len(self.V[v1]['neighbours'])
        else:
            return None
        
            
    
    def clear_coloring(self):
        self.used_colors = set()
        self.coloring = dict()

        
    def recolor(self):
        """
        Использовать с candidates при ветвлении
        т.к в первоначальной окраске слишком много цветов
        скорее всего стоит красить не каждое ветвление, а мб только в начале
        """
    #     print('***recolor***')
        max_color = -1
        self.coloring = dict()
        for vertex_name in self.V:
            self.coloring[vertex_name] = None
        self.used_colors = dict()
        for vertex_name in self.V:
#             print('vertex_name',vertex_name)
#             print('used_Colors',self.used_colors)
            avalible_colors = set(self.used_colors.keys() )
            for neighbour_name in g.V[vertex_name]['neighbours']:
#                 print('N',neighbour_name, self.coloring[neighbour_name] ,)
                avalible_colors -= {self.coloring[neighbour_name]}
                if len( avalible_colors ) == 0:
                    break
            if len( avalible_colors ) == 0:
                max_color += 1 # color is index in candidates_coloring
                self.used_colors[max_color] = set()
                self.used_colors[max_color].add(vertex_name)
                self.coloring[vertex_name] = max_color

#             print('avalible_colors',avalible_colors)
            if len( avalible_colors ) != 0:
                for avalible_color in avalible_colors:
                    rand_avalible_color = avalible_color
                    break
                self.used_colors[rand_avalible_color].add(vertex_name)
                self.coloring[vertex_name] = rand_avalible_color
#             print('chosen_Color',self.coloring[vertex_name])
#         return used_colors,candidates_coloring

    def recolor_by_degree(self, rev = False):
            """
            Использовать с candidates при ветвлении
            т.к в первоначальной окраске слишком много цветов
            скорее всего стоит красить не каждое ветвление, а мб только в начале
            """
        #     print('***recolor***')    
            self.sort_vertex_by_degree()
            max_color = -1
            self.coloring = dict()
            for vertex_name in self.V:
                self.coloring[vertex_name] = None
            self.used_colors = dict()
            if rev:
                D_range = range(len(self.D)-1,-1,-1)
            else:
                D_range = range(len(self.D))
            for vertex_index in D_range:
                vertex = self.D[vertex_index]
                vertex_name = vertex['name']
    #             print('vertex_name',vertex_name)
    #             print('used_Colors',self.used_colors)
                avalible_colors = set(self.used_colors.keys() )
                for neighbour_name in g.V[vertex_name]['neighbours']:
    #                 print('N',neighbour_name, self.coloring[neighbour_name] ,)
                    avalible_colors -= {self.coloring[neighbour_name]}
                    if len( avalible_colors ) == 0:
                        break
                if len( avalible_colors ) == 0:
                    max_color += 1 # color is index in candidates_coloring
                    self.used_colors[max_color] = set()
                    self.used_colors[max_color].add(vertex_name)
                    self.coloring[vertex_name] = max_color

    #             print('avalible_colors',avalible_colors)
                if len( avalible_colors ) != 0:
                    for avalible_color in avalible_colors:
                        rand_avalible_color = avalible_color
                        break
                    self.used_colors[rand_avalible_color].add(vertex_name)
                    self.coloring[vertex_name] = rand_avalible_color
    #             print('chosen_Color',self.coloring[vertex_name])
    
    def recolor_best_method(self):
        self.recolor()
        c1 = len(self.used_colors)
        self.clear_coloring()

        self.recolor_by_degree(rev = False)
        c2 = len(self.used_colors)
        self.clear_coloring()

        self.recolor_by_degree(rev = True)
        c3 = len(self.used_colors)
        self.clear_coloring()

        print('g.coloring()', c1)
        print('g.coloring_by_degree()', c2)
        print('g.coloring_by_degree_reverse()', c3)


        if c1 is min(c1,c2,c3):
            self.recolor()

        if c2 is min(c1,c2,c3):
            self.recolor_by_degree(rev = False)
        if c3 is min(c1,c2,c3):
            self.recolor_by_degree(rev = True)

        print('best g.colors', len(g.used_colors))
        


In [3]:
def read_dimacs_graph(file_path):
    '''
        Parse .col file and return graph object
    '''
    g = Graph()
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('c'):  # graph description
                print(*line.split()[1:])
            # first line: p name num_of_vertices num_of_edges
            elif line.startswith('p'):
                p, name, vertices_num, edges_num = line.split()
                print('{0} {1} {2}'.format(name, vertices_num, edges_num))
            elif line.startswith('e'):
                _, v1, v2 = line.split()
                g.add_edge(int(v1),int(v2))
            else:
                continue
        return g


In [4]:
def candidates_recolor(candidates:set):
    """
    Использовать с candidates при ветвлении
    т.к в первоначальной окраске слишком много цветов
    скорее всего стоит красить не каждое ветвление, а мб только в начале
    """
#     print('***candidates_recolor***')
    max_color = 0
    candidates_coloring = dict()
    for vertex_name in candidates:
        candidates_coloring[vertex_name] = 0
    used_colors = {0: set()}
    for vertex_name in candidates:
#         print('vertex_name',vertex_name)
#         print('used_Colors',used_colors)
        neighbour_colors = set()
        for neighbour_name in g.V[vertex_name]['neighbours'].intersection(candidates):
#             print('N',neighbour_name, candidates_coloring[neighbour_name] , neighbour_name in candidates)
            neighbour_colors.add(candidates_coloring[neighbour_name])
            if len( neighbour_colors ) == len(used_colors):
#                 print('break')
                break
        if len( neighbour_colors ) == len(used_colors):
            max_color += 1 # color is index in candidates_coloring
            used_colors[max_color] = set()
            used_colors[max_color].add(vertex_name)
            candidates_coloring[vertex_name] = max_color
#             print(used_colors,candidates_coloring)
        
#         print('avalible_colors',avalible_colors)
        elif len( neighbour_colors ) < len(used_colors):
            avalible_colors = set(used_colors.keys()) - neighbour_colors
#             print('avalible_colors',avalible_colors)
            for avalible_color in avalible_colors:
                rand_avalible_color = avalible_color
                break
            used_colors[rand_avalible_color].add(vertex_name)
            candidates_coloring[vertex_name] = rand_avalible_color
#         print('chosen_Color',candidates_coloring[vertex_name])
    if len(candidates_coloring) != len(candidates):
        raise NameError('Not all verticies colored')
    return used_colors,candidates_coloring

def candidates_recolor_degree(candidates:set, rev = False):
    """
    Использовать с candidates при ветвлении
    т.к в первоначальной окраске слишком много цветов
    скорее всего стоит красить не каждое ветвление, а мб только в начале
    """
#     print('***candidates_recolor
    candidates_degree_order = list(candidates)
    candidates_degree_order.sort(key= lambda input: g.V[input]['degree'], reverse = rev)
    max_color = -1
    candidates_coloring = dict()
    for vertex_name in candidates:
        candidates_coloring[vertex_name] = None
    used_colors = dict()
    for vertex_name in candidates_degree_order:
#         print('vertex_name',vertex_name)
#         print('used_Colors',used_colors)
        avalible_colors = set(used_colors.keys() )
        for neighbour_name in g.V[vertex_name]['neighbours'].intersection(candidates):
#             print('N',neighbour_name, candidates_coloring[neighbour_name] , neighbour_name in candidates)
            avalible_colors -= {candidates_coloring[neighbour_name]}
            if len( avalible_colors ) == 0:
                break
        if len( avalible_colors ) == 0:
            max_color += 1 # color is index in candidates_coloring
            used_colors[max_color] = set()
            used_colors[max_color].add(vertex_name)
            candidates_coloring[vertex_name] = max_color
        
#         print('avalible_colors',avalible_colors)
        if len( avalible_colors ) != 0:
            for avalible_color in avalible_colors:
                rand_avalible_color = avalible_color
                break
            used_colors[rand_avalible_color].add(vertex_name)
            candidates_coloring[vertex_name] = rand_avalible_color
#         print('chosen_Color',candidates_coloring[vertex_name])
    return used_colors,candidates_coloring

In [51]:
def find_candidates(vertex_to_clique = None):
    """
    если клика пустая создает множество из всех вершин графа
    если не пустая выдает ошибку - использовать update_candidates
    """
    global clique, candidates
    
    if len(clique) == 0:
        candidates = set( g.V.keys() )
    else:
        raise NameError('len(clique) =! 0. Use update_candidates')

    return candidates

In [6]:
def update_candidates(used_candidates = None, candidates = None, vertex_to_clique = None):
    """
    если клика пустая выдает ошибку - нужно использовать find_candidates
    если клика не пустая то обновляет множетсво кандидитов:
        в него включаются те вершины, которые есть в кандидатах
        и те вершины, которые есть в соседях у вершины vertex_to_clique, которая
        в данный момент добавляется в клику
    """
    global clique #, candidates
    
    if len(clique) == 0:
        raise NameError('len(clique) == 0. Use find_candidates')
    if candidates is None:
        raise NameError('candidates = None')
    if used_candidates is None:
        used_candidates = set()
    else:
        if vertex_to_clique not in candidates:
            raise NameError(vertex_to_clique,'No such vertex in candidates')
        updated_candidates = candidates.intersection(g.V[vertex_to_clique]['neighbours']) - used_candidates
    return updated_candidates

In [7]:
def initial_heuristic_degree():
    clique = set()
    g.sort_vertex_by_degree()
#     print(type(clique))
    for vertex in g.D:
#         print(type(clique),type(vertex))
        
        all_elements_in_clique_are_neighbours_to_vertex = True
        for element in clique:
            all_elements_in_clique_are_neighbours_to_vertex = all_elements_in_clique_are_neighbours_to_vertex and g.is_neighbour(vertex['name'],element)
            if not all_elements_in_clique_are_neighbours_to_vertex:
                break
#         print(vertex['name'],all_elements_in_clique_are_neighbours_to_vertex )
        if all_elements_in_clique_are_neighbours_to_vertex:
            clique.add(vertex['name'])
    return clique

In [8]:
def initial_heuristic_color(rev = False):
    """
    makes clique:
    first - vericies with maximal color
    """
    clique = set()
    g.sort_vertex_by_degree()
    if rev:
        color_range =  range(len(g.used_colors)-1,-1,-1)
    else:
        color_range =  range(len(g.used_colors))
    for color in color_range:
        for vertex_name in g.used_colors[color]:
            all_elements_in_clique_are_neighbours_to_vertex = True
            for element_name in clique:
                all_elements_in_clique_are_neighbours_to_vertex = all_elements_in_clique_are_neighbours_to_vertex and g.is_neighbour(vertex_name,element_name)
                if not all_elements_in_clique_are_neighbours_to_vertex:
                    break
#             print(vertex_name,all_elements_in_clique_are_neighbours_to_vertex )
            if all_elements_in_clique_are_neighbours_to_vertex:
                clique.add(vertex_name)
    return clique

In [9]:
def find_best_heuristic():
    init_clique = initial_heuristic_color()
    init_clique_rev = initial_heuristic_color(True)
    init_clique_degree = initial_heuristic_degree()
    len_best_heuristic =  max( len(init_clique),len(init_clique_rev),len(init_clique_degree)  )
    return max(init_clique,init_clique_rev,init_clique_degree),len_best_heuristic

In [10]:
def find_best_heuristic_new():
    g.clear_coloring()
    g.recolor()
    init_clique_1 = initial_heuristic_color()
    init_clique_rev_1 = initial_heuristic_color(True)
    init_clique_degree_1 = initial_heuristic_degree()
    len_best_heuristic_1 =  max( len(init_clique_1),len(init_clique_rev_1),len(init_clique_degree_1)  )
    best_heuristic_1 = max(init_clique_1,init_clique_rev_1,init_clique_degree_1)
    
    g.clear_coloring()
    g.recolor_by_degree()
    init_clique_2 = initial_heuristic_color()
    init_clique_rev_2 = initial_heuristic_color(True)
    init_clique_degree_2 = initial_heuristic_degree()
    len_best_heuristic_2 =  max( len(init_clique_2),len(init_clique_rev_2),len(init_clique_degree_2)  )
    best_heuristic_2 = max(init_clique_2,init_clique_rev_2,init_clique_degree_2)
    
    g.clear_coloring()
    g.recolor_by_degree(rev = True)
    init_clique_3 = initial_heuristic_color()
    init_clique_rev_3 = initial_heuristic_color(True)
    init_clique_degree_3 = initial_heuristic_degree()
    len_best_heuristic_3 =  max( len(init_clique_3),len(init_clique_rev_3),len(init_clique_degree_3)  )
    best_heuristic_3 = max(init_clique_3,init_clique_rev_3,init_clique_degree_3)
    
    
    len_best_heuristic =  max(len_best_heuristic_1, len_best_heuristic_2, len_best_heuristic_3)
    return max(best_heuristic_1,best_heuristic_2,best_heuristic_3),len_best_heuristic

In [11]:
path_e_1_1 = 'test/brock200_1.clq.txt'
path_e_1_2 = 'test/brock200_2.clq.txt'
path_e_1_3 = 'test/brock200_3.clq.txt'
path_e_1_4 = 'test/brock200_4.clq.txt'
path_e_2_1 = 'test/gen200_p0.9_44.clq.txt'
path_e_2_2 = 'test/gen200_p0.9_55.clq.txt'
path_e_3 = 'test/hamming8-4.clq.txt'
path_e_4 = 'test/johnson16-2-4.clq.txt'
path_e_5 = 'test/keller4.clq.txt'
path_e_6 = 'test/MANN_a27.clq.txt'
path_e_7 = 'test/p_hat1000-1.clq.txt'

# Графовые функции branch_and_cut 

In [14]:
def find_normed_degree_dict(g):
    """
    for veretx with the lowes degree - 1
    for veretx with the highest degree - 0
    
    made to find vertexes with lowest degree
    for use in find_most_violated_greedy_and_degree
    
    """
    degree_arr = []
    for vertex_name in g.V:
        (vertex_name - 1, g.V[vertex_name]['degree'])
        degree_arr.append( (vertex_name -1 , g.V[vertex_name]['degree'])  ) # degree_arr - array of tupples (vertex_index,vertex_deg)
    degree_arr.sort(key = lambda i: i[1], reverse = False)
    
    min_degree = degree_arr[0][1]
    max_degree = degree_arr[-1][1]
    
    normed_degree_dict = dict()
    for index in range(len(degree_arr)):
        vertex_index = degree_arr[index][0]
        degree = degree_arr[index][1]
        normed_degree = 1 - (degree-min_degree)/(max_degree-min_degree)

        normed_degree_dict[vertex_index] = normed_degree
    return normed_degree_dict

In [15]:
def find_independent_sets_by_coloring(coloring_colors, vertex_coloring, candidates):
    global g
    colors = copy.deepcopy(coloring_colors)
    ind_sets_coloring = dict()
    for vertex_name in vertex_coloring:
        ind_sets_coloring[vertex_name] = {vertex_coloring[vertex_name]}
    for color in colors:
        for vertex_name in coloring_colors[color]:
            available_colors = set(colors.keys())
            available_neighbours = g.V[vertex_name]['neighbours'].intersection(candidates)
            while len(available_neighbours) > 0:
                neighbour = available_neighbours.pop()
                neighbour_color = vertex_coloring[neighbour]
                neighbour_colors_set = ind_sets_coloring[neighbour]
                available_colors -= {neighbour_color}.union(neighbour_colors_set)
                for n_color in neighbour_colors_set:
                    available_neighbours -= colors[n_color]
            check_neighbours = set()
            # есть цвета по которым может быть пересечение. Их надо проверить
            for available_color in available_colors:
                available_neighbours = g.V[vertex_name]['neighbours'].intersection(candidates)
                available_neighbours = available_neighbours.intersection(
                    colors[available_color])  # change to colors
                check_neighbours = check_neighbours.union(available_neighbours)
            for neighbour in check_neighbours:
                available_colors -= ind_sets_coloring[neighbour]
            for available_color in available_colors:
                colors[available_color].add(vertex_name)
            ind_sets_coloring[vertex_name] = ind_sets_coloring[vertex_name].union(available_colors)
    return colors, ind_sets_coloring

# CPLEX функции branch_and_cut 

In [17]:
def ind_sets_most_violated(ind_set: set):
    """
    set is set of indexes
    """

    for vertex_index_1 in ind_set:
        for vertex_index_2 in ind_set:
            if vertex_index_1 is not vertex_index_2:
                vertex_name_1 = vertex_index_1 + 1
                vertex_name_2 = vertex_index_2 + 1
                if  g.is_neighbour(vertex_name_1,vertex_name_2):
                    print(g.is_neighbour(vertex_name_1,vertex_name_2))
    print('if pusto - OK')

In [868]:
def find_most_violated_greedy_and_degree_maybe_new(random_range = 4, sum_without_random = 5, expandable_set = None):
    """
    Heuristical algorithm to find_most_violated_greedy_and_degree
    
    random_range - number of iterations with randomes(alpha,betta)
    sum_without_random - if summ if vertex variable more than sum_without_random,
    algorithm don`t use random iterations
    
    best_ind_set - set of vertex_indexes 
    best_sum - sum of variables for best_ind_set
    """
    global x,mdl, normed_degree_dict, x_vars
    alpha_arr = [1]
    betta_arr = [0]
    best_sum = 0
    if expandable_set is not None:
             best_sum = -1
    best_ind_set = {}
    
    for i in range(random_range):
        alpha_arr.append(random.uniform(0,1))
        betta_arr.append(random.uniform(0,1))
    for alpha, betta in zip(alpha_arr,betta_arr):
        sum_x = 0
        # x_vars - array of tupples (vertex_index,vertex_variable)
        # sol.iter_var_values() - iteration by nonzero variable_values
        x_vars = [(vertex_variable.get_index(),vertex_variable) for (vertex_variable,vertex_variable_val) in sol.iter_var_values()]
        x_vars.sort(key = lambda i: alpha * i[1].solution_value + betta * normed_degree_dict[i[0]], reverse = True)#sort partly by degree and by solution valuae
        if expandable_set is None:
            ind_set = set()
            ind_set.add (x_vars[0][0])
        else:
            ind_set = expandable_set
        for vertex_index_1,vertex_variable in x_vars: # index = vertex_name - 1
            addition_forbidden = False
            for vertex_index_2 in ind_set:
                vertex_name_1 = vertex_index_1 + 1
                vertex_name_2 = vertex_index_2 + 1
                if g.is_neighbour(vertex_name_1,vertex_name_2):
                    addition_forbidden = True
                    break
            if not addition_forbidden:
                ind_set.add (vertex_index_1)
                sum_x += vertex_variable.solution_value
        if sum_x > best_sum:
            best_sum = sum_x
            best_ind_set = ind_set
        if sum_x > sum_without_random:
            break
    return best_ind_set, best_sum

In [486]:
def find_most_violated_greedy_and_degree(random_range = 4, sum_without_random = 5):
    """
    Heuristical algorithm to find_most_violated_greedy_and_degree
    
    random_range - number of iterations with randomes(alpha,betta)
    sum_without_random - if summ if vertex variable more than sum_without_random,
    algorithm don`t use random iterations
    
    best_ind_set - set of vertex_indexes 
    best_sum - sum of variables for best_ind_set
    """
    global x,mdl, normed_degree_dict, x_vars
    alpha_arr = [1]
    betta_arr = [0]
    best_sum = 0
    best_ind_set = {}
    
    for i in range(random_range):
        alpha_arr.append(random.uniform(0,1))
        betta_arr.append(random.uniform(0,1))
    for alpha, betta in zip(alpha_arr,betta_arr):
        sum_x = 0
        # x_vars - array of tupples (vertex_index,vertex_variable)
        # sol.iter_var_values() - iteration by nonzero variable_values
        x_vars = [(vertex_variable.get_index(),vertex_variable) for (vertex_variable,vertex_variable_val) in sol.iter_var_values()]
        x_vars.sort(key = lambda i: alpha * i[1].solution_value + betta * normed_degree_dict[i[0]], reverse = True)#sort partly by degree and by solution valuae
        ind_set = set()
        ind_set.add (x_vars[0][0])
        for vertex_index_1,vertex_variable in x_vars: # index = vertex_name - 1
            addition_forbidden = False
            for vertex_index_2 in ind_set:
                vertex_name_1 = vertex_index_1 + 1
                vertex_name_2 = vertex_index_2 + 1
                if g.is_neighbour(vertex_name_1,vertex_name_2):
                    addition_forbidden = True
                    break
            if not addition_forbidden:
                ind_set.add (vertex_index_1)
                sum_x += vertex_variable.solution_value
        if sum_x > best_sum:
            best_sum = sum_x
            best_ind_set = ind_set
        if sum_x > sum_without_random:
            break
    return best_ind_set, best_sum

In [112]:
def build_problem():
    """
    makes new global mdl - model, x - variables of model
    by existing global graph g 
    """
    
    global mdl,x,g
    mdl = Model(name='max clique')
    x_range = range(len(g.V))
    # decision variables
    # x = mdl.integer_var_dict(x_range)
    x = mdl.continuous_var_dict(x_range)
    ind_sets_arr = []
    clique = set() # for find_candidates()
    candidates = find_candidates()
    (colors, vertex_coloring) = candidates_recolor(candidates)
    ind_sets_arr.append(find_independent_sets_by_coloring(colors, vertex_coloring, candidates)[0])
    (colors, vertex_coloring) = candidates_recolor_degree(candidates)
    ind_sets_arr.append(find_independent_sets_by_coloring(colors, vertex_coloring, candidates)[0])
    (colors, vertex_coloring) = candidates_recolor_degree(candidates, rev=True)
    ind_sets_arr.append(find_independent_sets_by_coloring(colors, vertex_coloring, candidates)[0])

    for ind_sets in ind_sets_arr:
        for ind_set in ind_sets:
            # print(i, colors2[i])
    #         if len(ind_sets[ind_set]) > 1:
            mdl.add_constraint(mdl.sum(x[vertex_name-1] for vertex_name in ind_sets[ind_set]) <= 1) #vertex_index = vertex_name-1

#     for i in range(mdl.number_of_constraints):
#         print(mdl.get_constraint_by_index(i))

    for i in x_range:
#         mdl.add_constraint(x[i] <= 1)
        mdl.add_constraint(x[i] >= 0)

    mdl.maximize(mdl.sum(x[i] for i in x_range))

In [133]:
def is_int_value(value):
    """
    True - if int
    False if float
    
    """
    
    if value % 1 > eps and abs(value % 1 - 1) > eps:
        return False
    return True


In [683]:
def is_int_solution():
    """
    True - if all variables int
    False if even one of variables is float
    """
    global sol
    
    sol_val = sol.get_objective_value()
    if not is_int_value(sol_val):
        return False
    for var in sol.iter_var_values():
        if not is_int_value(var[1]):
            return False
    return True

In [755]:
def is_a_clique():
    """
    check solution if is_int_solution() is True
    
    function check if varticies vith x = 1 are clique or not
    """
    global sol, x_vars,g
    
    vertex_names = [i[0].index+1 for i in sol.iter_var_values()] #заменить на x_vars уточнить 
    for vertex_name_1 in vertex_names:
        for vertex_name_2 in vertex_names:
            if vertex_name_1 is not vertex_name_2:
                if not g.is_neighbour(vertex_name_1,vertex_name_2):
                    print(f"{vertex_name_1,vertex_name_2} are not neighbours. Not a clique")
                    return False
    return True
    

In [ ]:
def add_non_edges_constraints_maybe_new():
    """
    work if  is_a_clique() is False
    function add new constraints for if varticies vith x = 1 if they are not neighbours
    """
    global sol, x_vars,g
#     non_edges = []
    vertex_indexes = [i[0].index for i in sol.iter_var_values()]
    for vertex_index_1 in vertex_indexes:
        for vertex_index_2 in vertex_indexes:
            if vertex_index_1 is not vertex_index_2:
                vertex_name_1 = vertex_index_1 + 1
                vertex_name_2 = vertex_index_2 + 1
                if not g.is_neighbour(vertex_name_1,vertex_name_2):
                    ind_set = {vertex_name_1,vertex_name_2}
                    
                    con = x[vertex_index_1] + x[vertex_index_2] <= 1
                    mdl.add_constraint(con)
                    
                    print(f"{vertex_name_1,vertex_name_2} are not neighbours. Added {con}")



In [881]:
def add_non_edges_constraints():
    """
    work if  is_a_clique() is False
    function add new constraints for if varticies vith x = 1 if they are not neighbours
    """
    global sol, x_vars,g
#     non_edges = []
    vertex_names = [i[0].index+1 for i in sol.iter_var_values()]
    for vertex_name_1 in vertex_names:
        for vertex_name_2 in vertex_names:
            if vertex_name_1 is not vertex_name_2:
                if not g.is_neighbour(vertex_name_1,vertex_name_2):
                    vertex_index_1 = vertex_name_1 - 1
                    vertex_index_2 = vertex_name_2 - 1
                    con = x[vertex_index_1] + x[vertex_index_2] <= 1
                    mdl.add_constraint(con)
                    
                    print(f"{vertex_name_1,vertex_name_2} are not neighbours. Added {con}")



In [23]:
def get_non_edges_old(sol, g=Graph()):
    non_edges = []
    var_values = [i[0] for i in sol.iter_var_values()]
    for i in range(len(var_values)):
        for j in range(i+1, len(var_values)):
            if var_values[j].index+1 not in g.V[var_values[i].index+1]['neighbours']:
                non_edges.append((var_values[j], var_values[i]))
    return non_edges


In [709]:
def branch_and_cut_choose_branching_var():
    """
    x_vars - sort partly by degree and by solution valuae f
    x_vars from find_most_violated_greedy_and_degree
    algorithm choose first available float variable from sorted x_vars 
    return fraction_variable_index
    """
    global len_max_current_clique, sol, x_vars

    ind_set, x_sum = find_most_violated_greedy_and_degree(sum_without_random = 0)    # to sort x_vars by var_value
    if len(ind_set) > 2:
        mdl.add_constraint(mdl.sum(x[i] for i in ind_set) <= 1)    # to sort x_vars by var_value
        sol = mdl.solve()    # to sort x_vars by var_value
    
    for vertex_index, vertex_var in x_vars:
        if not is_int_value(vertex_var.solution_value):
            fraction_variable_index = vertex_index
            break
    return fraction_variable_index

In [927]:
def branch_and_cut_search(sum_without_random = 2):
    global len_max_current_clique, sol, x_vars
    sol = mdl.solve() # нужно ли тут
    if sol is None:
        return
    
    if sol.get_objective_value() <= len_max_current_clique:
        return
    
    
    first_time = True # to do first iteration on while every time
    
    while first_time or x_sum > 1 :
        ind_set, x_sum = find_most_violated_greedy_and_degree(sum_without_random)
        if len(ind_set) <= 1:
            break
        most_violated_con = mdl.sum(x[i] for i in ind_set) <= 1
        mdl.add_constraint(most_violated_con)
        sol = mdl.solve()
        
        if sol is None:
            return
        if  is_int_solution():
            break
        first_time = False
        
    if not is_int_solution():
        fraction_variable_index = branch_and_cut_choose_branching_var()
        con1 = x[fraction_variable_index] >= 1
        con2 = x[fraction_variable_index] <= 0
        mdl.add_constraint(con1)
        branch_and_cut_search(sum_without_random)
        mdl.remove_constraint(con1)
        
        mdl.add_constraint(con2)
        branch_and_cut_search(sum_without_random)
        mdl.remove_constraint(con2)
    else: # solution is int 
        if is_a_clique():
            current_clique = sol.get_objective_value()
            if current_clique > len_max_current_clique:     
                len_max_current_clique = current_clique
                print(f"current_len_max_clique is {len_max_current_clique}")
                print(f"current_max_clique is:")
                mdl.print_solution()
        else:
            add_non_edges_constraints()
    
    
    
        

# Algorithm

In [427]:
path_e_1_1 = 'test/brock200_1.clq.txt'
path_e_1_2 = 'test/brock200_2.clq.txt'
path_e_1_3 = 'test/brock200_3.clq.txt'
path_e_1_4 = 'test/brock200_4.clq.txt'
path_e_2_1 = 'test/gen200_p0.9_44.clq.txt'
path_e_2_2 = 'test/gen200_p0.9_55.clq.txt'
path_e_3 = 'test/hamming8-4.clq.txt'
path_e_4 = 'test/johnson16-2-4.clq.txt'
path_e_5 = 'test/keller4.clq.txt'
path_e_6 = 'test/MANN_a27.clq.txt'
path_e_7 = 'test/p_hat1000-1.clq.txt'

In [928]:
g = read_dimacs_graph(path_e_1_2);

FILE: brock200_2.b

Graph Generator

By: Mark Brockington (brock@cs.ualberta.ca)
and Joe Culberson (joe@cs.ualberta.ca)

Graph Size:200, Clique Size: 12
Seed:20, Edge Density: 0.50000
Depth 1 Hiding

Clique Elements are:
26 120 119 157 69 182 47 148 104 134
54 144

Estimated Size of Uncompressed File: 0.1MB

edge 200 9876


In [929]:
normed_degree_dict = find_normed_degree_dict(g)

In [930]:
_,len_max_current_clique = find_best_heuristic_new()
print(len_max_current_clique)

8


In [931]:
clique = set() # for build_problem()
x_vars = 0
sol = 0

In [932]:
build_problem()

In [933]:
len(x)

200

In [934]:
sol = mdl.solve()

In [935]:
branch_and_cut_search(sum_without_random = 2)

(121, 173) are not neighbours. Not a clique
(121, 173) are not neighbours. Added x121+x173 <= 1
(144, 145) are not neighbours. Added x144+x145 <= 1
(144, 178) are not neighbours. Added x144+x178 <= 1
(145, 144) are not neighbours. Added x144+x145 <= 1
(173, 121) are not neighbours. Added x121+x173 <= 1
(178, 144) are not neighbours. Added x144+x178 <= 1
(103, 112) are not neighbours. Not a clique
(103, 112) are not neighbours. Added x103+x112 <= 1
(103, 138) are not neighbours. Added x103+x138 <= 1
(112, 103) are not neighbours. Added x103+x112 <= 1
(134, 138) are not neighbours. Added x134+x138 <= 1
(134, 151) are not neighbours. Added x134+x151 <= 1
(138, 103) are not neighbours. Added x103+x138 <= 1
(138, 134) are not neighbours. Added x134+x138 <= 1
(138, 194) are not neighbours. Added x138+x194 <= 1
(151, 134) are not neighbours. Added x134+x151 <= 1
(194, 138) are not neighbours. Added x138+x194 <= 1
(67, 134) are not neighbours. Not a clique
(67, 134) are not neighbours. Added x

current_len_max_clique is 12.0
current_max_clique is:
objective: 12.000
  x27=1.000
  x48=1.000
  x55=1.000
  x70=1.000
  x105=1.000
  x120=1.000
  x121=1.000
  x135=1.000
  x145=1.000
  x149=1.000
  x158=1.000
  x183=1.000


## замеры, проверки

In [16]:
def ind_sets_by_coloring_check(ind_sets:dict):


    for ind_set in ind_sets:
        for vertex_1 in ind_sets[ind_set]:
            for vertex_2 in ind_sets[ind_set]:
                if vertex_1 is not vertex_2:
                    if  g.is_neighbour(vertex_2,vertex_1):
                        print(g.is_neighbour(vertex_2,vertex_1))
    print('if pusto - OK')

In [924]:
for index, var in x_vars:
    print(index, var.solution_value)

45 0
74 0.1495133346503552
148 0.15327905139598283
73 0.09131098411449645
172 0.17822932413211243
115 0.18473417760865987
79 0.15038081123526859
144 0.08660429093985866
143 0.16956639629582856
142 0.20149713260344965
194 0.08681803336466681
102 0.26231151836345135
174 0.22860213725431008
18 0.06896602805818935
11 0.12234656248744513
25 0.2038812208837038
10 0.19661910317067466
113 0.12483510386731873
22 0.18518668643606281
70 0.1124648617332554
191 0.10564725728718914
38 0.12208615958921731
195 0.10931304033617448
41 0.18942428739960962
100 0.1409373224651493
85 0.10091060669205165
171 0.1435240217509062
186 0.14225344229999373
76 0.11809047745214193
125 0.1361987993243135
64 0.15483821558863084
121 0.20046317248892379
129 0.11463510201824753
137 0.13423983251034047
110 0.010368830196651718
119 0.21424815820900145
89 0.14763885248059885
54 0.1547354122942093
122 0.0831813719286087
187 0.128526651398991
164 0.18238431949824246
141 0.10396756392740275
124 0.09984342007190208
30 0.2069355

In [601]:
branch_and_cut_choose_branching_var()

21

In [603]:
x[21].solution_value

0.38344893861754814

In [784]:
mdl.number_of_constraints

620

In [802]:
i = 0
for i in range(mdl.number_of_constraints):
    print(mdl.get_constraint_by_index(i))
    i += 1

x1+x3+x68+x192+x200 <= 1
x1+x2+x38+x97+x184 <= 1
x3+x6+x14+x77 <= 1
x4+x7+x10+x61 <= 1
x5+x17+x24+x49 <= 1
x8+x9+x64+x87 <= 1
x11+x15+x58+x113+x170 <= 1
x1+x12+x19+x25+x196 <= 1
x13+x21+x35 <= 1
x16+x18+x30+x100 <= 1
x20+x22+x40+x171+x180 <= 1
x23+x30+x32+x39 <= 1
x26+x27+x43+x140 <= 1
x2+x28+x37+x95+x196 <= 1
x29+x31+x34+x108 <= 1
x33+x41+x69+x117 <= 1
x36+x42+x47+x65 <= 1
x2+x28+x44+x46+x70 <= 1
x45+x48+x56+x133 <= 1
x50+x57+x60+x68+x163 <= 1
x51+x53+x54+x91+x168+x180 <= 1
x7+x52+x62+x71 <= 1
x33+x55+x63+x67+x146 <= 1
x59+x66+x75+x200 <= 1
x25+x72+x73+x95+x183 <= 1
x74+x76+x98+x110+x129+x193 <= 1
x78+x79+x94+x147 <= 1
x5+x80+x85+x88+x194 <= 1
x81+x84+x107+x123 <= 1
x82+x83+x109 <= 1
x70+x86+x89+x121+x137 <= 1
x43+x90+x169+x190 <= 1
x1+x92+x99+x101+x118 <= 1
x93+x105+x127+x138 <= 1
x38+x96+x112+x119 <= 1
x102+x104+x111+x116 <= 1
x103+x113+x136+x174 <= 1
x106+x122+x124+x135+x159 <= 1
x51+x114+x115 <= 1
x120+x150+x152+x161 <= 1
x40+x125+x126+x128+x142 <= 1
x43+x130+x131+x164 <= 1
x132+x

x2+x42+x111

In [627]:
x[14].solution_value

DOcplexException: Model<max clique> did not solve successfully

##def is_a_clique():
x_vars == vertex_names? вплане смысла 